# PR-2 Image_Segmentation 

In [ ]:
import numpy as np

Mount data from google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_images(path,_type):
    if _type == "test":
      test_images = []
    elif _type == "train":
      train_images = []
    else:
      val_images = []

In [ ]:
def visualize(img):
  '''
  reads an image and display 
  with its associated ground truth segmentation(s).
  '''
  pass

In [ ]:
def 